In [1]:
using Plots
using TensorCrossInterpolation, LinearAlgebra
using QuanticsTCI
using QuanticsGrids
using ITensorMPS
using ITensors
using NDTensors
using TCIITensorConversion
include("src/differential_mpo.jl")

Diff_2_2_y (generic function with 1 method)

In [117]:
R = 10
tol = 1e-8
max_bond = 30
delta_t = 1
v = 1

a = 1e1
xmin = -5
xmax = 5
ymin = -5
ymax = 5
dx = float(abs(xmin - xmax) / 2^R)
dy = float(abs(ymin - ymax) / 2^R)

function u_1(x,y)
    return exp(-((x/a)^2/2) + ((y/a)^2)/2)
end
function u_2(x,y)
    return exp(-((x/a)^2/2) + ((y/a)^2)/2)
end

u_2 (generic function with 1 method)

In [ ]:
# QuanticsTCI
qgrid = DiscretizedGrid{2}(R, (xmin, ymin), (xmax, ymax); includeendpoint = true)
u1q, ranks1, errors1 = quanticscrossinterpolate(Float64, u_1, qgrid; tolerance=tol)
u2q, ranks2, errors2 = quanticscrossinterpolate(Float64, u_2, qgrid; tolerance=tol)

# Convert to MPS Format
s = siteinds("Qudit", 10, dim=4)
u = [ITensorMPS.MPS(TensorTrain(u1q.tci), sites=s), ITensorMPS.MPS(TensorTrain(u2q.tci), sites=s)]

# Build Diff MPOs
d1 = [Diff_1_2_x(dx, s), Diff_1_2_y(dy, s)]
d2 = [Diff_2_2_x(dx, s), Diff_2_2_y(dy, s)]
print("Done!")

Done!

In [99]:
function apply_U_p_k_T(mps, u_c_p_k, center)
    result_l = ITensor(1.0)
    result_r = ITensor(1.0)

    for i in 1:center-1
        result_l *= dag(u_c_p_k[i]) * mps[i]
    end

    for i in length(s):-1:center+1
        result_r *= dag(u_c_p_k[i]) * mps[i]
    end

    return result_l * mps[center] * result_r
end

apply_U_p_k_T (generic function with 2 methods)

In [ ]:
function make_beta_k(u_c, u_c_p, k, center, delta_t, v, d1, d2, s)
    del = MPO([delta(s[i], s[i]', s[i]'') for i in 1:length(s)])
    u_c_del = [replaceprime(MPO(*(del, u_c[1]'')), 2=>1), replaceprime(MPO(*(del, u_c[2]'')), 2=>1)]
    u_k_d1 = [apply(d1[1], u_c[k], maxdim=max_bond), apply(d1[2], u_c[k], maxdim=max_bond)]
    u_k_del_d1 = [apply(u_c_del[1], u_k_d1[1], maxdim=maxlinkdim(u_k_d1[1])), apply(u_c_del[2], u_k_d1[2], maxdim=maxlinkdim(u_k_d1[2]))]
    u_k_d2 = [apply(d2[1], u_c[k], maxdim=max_bond), apply(d2[2], u_c[k], maxdim=max_bond)]

    result = apply_U_p_k_T(u_c[k], u_c_p[k], center)
    result += apply_U_p_k_T(-delta_t * u_k_del_d1[1], u_c_p[k], center)
    result += apply_U_p_k_T(-delta_t * u_k_del_d1[2], u_c_p[k], center)
    result += apply_U_p_k_T(delta_t * v * u_k_d2[1], u_c_p[k], center)
    result += apply_U_p_k_T(delta_t * v * u_k_d2[2], u_c_p[k], center)

    return result
end

make_beta_k (generic function with 1 method)

In [119]:
c_k = ITensor(u_c[1][center])

ITensor ord=3 (dim=4|id=282|"Qudit,Site,n=2") (dim=4|id=210|"l=1,link") (dim=13|id=207|"l=2,link")
Dense{Float64, Vector{Float64}}

In [123]:
filter(x->hastags(x, "link"), inds(c_k))

((dim=4|id=210|"l=1,link"), (dim=13|id=207|"l=2,link"))

In [ ]:
function apply_H_k_j(c_k, u_c_p, d1, center)
    c_k = replaceinds(c_k, filter(x->hastags(x, "link", "l=$(center)"), inds(c_k)), filter(x->hastags(x, "link"), inds(u_c_p[center])))
    u_c_p[center] = c_k

In [ ]:
center = 2
u_c = [orthogonalize(u[1], center), orthogonalize(u[2], center)]
u_c_p = [copy(u_c[1]), copy(u_c[2])]

beta = [make_beta_k(u_c, u_c_p, 1, center, delta_t, v, d1, d2, s), make_beta_k(u_c, u_c_p, 2, center, delta_t, v, d1, d2, s)]

print(beta[1])
print(beta[2])

ITensor ord=3
Dim 1: (dim=4|id=282|"Qudit,Site,n=2")
Dim 2: (dim=4|id=210|"l=1,link")
Dim 3: (dim=13|id=207|"l=2,link")
Dense{Float64, Vector{Float64}}
 4×4×13
[:, :, 1] =
 -2.5163637553061096e10  -2.2172291335071136e10  -2.3605176907009583e10  -2.3636150034505733e10
 -2.3647689204948128e10  -2.3601336446233166e10  -2.218311583766628e10   -2.5159543535390144e10
 -2.3616797155147606e10  -2.3601283614234062e10  -2.5126517903826847e10  -2.2183206211033188e10
 -2.219403604117754e10   -2.5122429926740967e10  -2.36128057606954e10    -2.3612954803999985e10

[:, :, 2] =
 -16787.70378704361         -1.112197883633853e9   -7.123624881093441e8  -4.7141710914575106e8
      2.3562062115370587e8  -9.480281792672186e8   -4.479689620813702e8  -2.5087088709607443e8
     -3.561460512076536e8   -1.5409404985633285e9  -1.137916772937574e9  -7.773864978746617e8
     -1.1332124770234293e8  -1.3889563320398157e9  -8.333800553360169e8  -5.9175367170603e8

[:, :, 3] =
 -11152.47351958964             2.21997513

In [ ]:


u1_c = orthogonalize(u[1], 4)
d1 = [Diff_1_2_x(dx, s_c), Diff_1_2_y(dy, s_c)]
del = MPO([delta(s_c[i], s_c[i]', s_c[i]'') for i in 1:length(s_c)])
u1_del = replaceprime(MPO(*(del, u1_c'')), 2=>1)
u1_d1_1 = apply(d1[1], u1_c, maxdim=max_bond)
u1_del_u1_d1_1 = apply(u1_del, u1_d1_1, maxdim=maxlinkdim(u1_d1_1))